In [1]:
import matplotlib.pyplot as plt
import operator
import pickle
import math

from matplotlib.font_manager import FontProperties

In [9]:
# 数据：银行是否给贷款

def createDataSet():
    dataSet = [
        [0, 0, 0, 0, 'no'],
        [0, 0, 0, 1, 'no'],
        [0, 1, 0, 1, 'yes'],
        [0, 1, 1, 0, 'yes'],
        [0, 0, 0, 0, 'no'],
        [1, 0, 0, 0, 'no'],
        [1, 0, 0, 1, 'no'],
        [1, 1, 1, 1, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [2, 0, 1, 1, 'yes'],
        [2, 1, 0, 1, 'yes'],
        [2, 1, 0, 2, 'yes'],
        [2, 0, 0, 0, 'no'],
    ]
    labels = ['F1-AGE', 'F2-WORK', 'F3-HOME', 'F4-LOAN']

    return dataSet, labels

In [3]:
def majorityCnt(classList):
    class_count = {}
    for vote in classList:
        if vote not in class_count.keys():
            class_count[vote] = 1
        else:
            class_count[vote] += 1
    # class_counted = sorted(class_count.items(), key=operator.itemgetter(1), reverse=True)
    class_counted = sorted(class_count.items(), key=lambda x: x[1], reverse=True)
    # class_counted 是一个 list，其中的每个元素是一个 (key, value) 元组
    
    # 或者可以：class_counted = sorted(class_count, key=lambda x:class_count[x], reverse=True)
    
    # return class_counted.keys()[0] 没法这样，因为class_counted是一个包含键值对的元组了已经
    return class_counted[0][0]

#### 熵值、信息增益的计算都在此函数中
注：  
1.dataset是一个列表，计算个数使用len；.shape是NumPy数组（或Pandas的DateFrame）的属性  

In [18]:
# 这是对分类结果的 信息熵
def calculate_ent(dataset):
    sum_number = len(dataset)
    label_count = {}
    for feat_vec in dataset:
        current_label = feat_vec[-1]
        if current_label not in label_count.keys():
            label_count[current_label] = 1
        else:
            label_count[current_label] += 1

    ent = 0
    for key in label_count:  # 不用keys()，默认就是对key
        prop = float(label_count[key]) / sum_number
        ent -= (prop * math.log(prop))

    return ent
        

# 切分数据集
def split_dataset(dataset, feat_val, value):
    sub_dataset = []
    for feat_vec in dataset:
        if feat_vec[feat_val] == value:
            # 每次迭代，要去掉此列
            # reduce_feat_vec = del feat_vec[feat_val]
            reduce_feat_vec = feat_vec[:feat_val] + feat_vec[feat_val+1:]
            sub_dataset.append(reduce_feat_vec)
    return sub_dataset


def choose_best_feature_to_split(dataset):
    num_feature = len(dataset[0]) - 1
    # ent不能设置为0，0相当于一开始是最好的
    ent = calculate_ent(dataset)
    best_information_gain = 0
    best_feat = None
    for i in range(num_feature):
        feat_list = [example[i] for example in dataset] # 拿到当前列
        unique_value = set(feat_list) # 类别个数
        gain = ent
        for value in unique_value:
            sub_dataset = split_dataset(dataset, i, value)
            prop = float(len(sub_dataset)) / float(len(feat_list))
            gain -= prop * calculate_ent(sub_dataset)
        if (gain > best_information_gain):
            best_information_gain = gain
            best_feat = i  # 是一个索引值
    return best_feat


### 是一个递归调用
#### 涉及到需要注意的点
1.list如何得到列长  
2.set()可以得到元素不重复的集合  

In [32]:
# 模型需要三个参数
# 递归调用，先判断是否是叶结点（内部的标签是否相同）

def createTree(dataset, labels, featLabels):
    classList = [example[-1] for example in dataset]

    # 判断：当前的是否是同一类别
    # classList[0] 代表的就是list里的第一个元素
    if classList.count(classList[0]) == len(classList):
        return classList[0]

    # 判断特征集是否为空集（选了一个特征，就得删除一个特征），是的话只需要返回其中类别最多的标签就行
    if len(dataset[0]) == 1:  # dataSet[0]就是第一行数据（二维列表，第一行就是第一个数据）
        return majorityCnt(classList)

    # 需要进一步分叉了
    bestFeat = choose_best_feature_to_split(dataset)  # 这是一个索引值
    best_feat_label = labels[bestFeat]
    featLabels.append(best_feat_label)

    # 树，对应一个嵌套的字典
    myTree = {best_feat_label:{}}

    # 删除此次使用的特征
    del labels[bestFeat]

    # 当前属性能分为几叉
    featValues = [example[bestFeat] for example in dataset]
    unique_vals = set(featValues) # 得到所有的可能值（集合，每个只出现一次）
    for value in unique_vals:
        sublabels = lables[:]
        # 对数进行下一步分叉，递归用函数 createTree
        myTree[best_feat_label][value] = createTree(split_dataset(dataset, bestFeat, value), sublabels, featLabels)

    return myTree

In [33]:
if __name__ == '__main__':
    dataset, labels = createDataSet()
    featLabels = []
    myTree = createTree(dataset, labels, featLabels)